In [2]:
from src.models.decisiontrees.xgboost import XGB1
from src.models.decisiontrees.catboost import CatBoost1
from src.models.decisiontrees.lightgbm import LGBM1
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
historical_metering = pd.read_csv("data/datasets2025/historical_metering_data_ES.csv")
rollout = pd.read_csv("data/datasets2025/rollout_data_ES.csv")

historical_copy = historical_metering.copy()
historical_copy['DATETIME'] = pd.to_datetime(historical_copy['DATETIME'])
historical_copy.set_index('DATETIME', inplace=True)
rollout_copy = rollout.copy()

In [4]:
historical_datetimes = historical_metering["DATETIME"]
rollout_datetimes = rollout["DATETIME"]
common_dates = set(historical_datetimes).intersection(rollout_datetimes)


In [5]:
historical_data = historical_metering[historical_metering["DATETIME"].isin(common_dates)].to_numpy()
rollout_data = rollout[rollout["DATETIME"].isin(common_dates)].to_numpy()
common_dates = list(common_dates)

In [6]:
# fig = plt.figure(figsize=(100, 20))
# plt.plot(common_dates, historical_data[:, 3], label="Historical Data")
# plt.plot(common_dates, rollout_data[:, 3], label="Rollout Data")
# plt.title("Historical and Rollout Data Comparison")
# plt.legend()
# plt.show()

In [7]:
# fig = plt.figure(figsize=(100, 20))
# plt.plot(common_dates, historical_data[:, 3]-rollout_data[:, 3], label="Error")
# plt.title("Historical and Rollout Data Comparison")
# plt.legend()
# plt.show()

In [8]:

# max_lag = len(historical_copy)
# lags = range(1,max_lag + 1)
# autocorrelations = [historical_copy['VALUEMWHMETERINGDATA_customerES_5'].autocorr(lag=lag) for lag in lags]
# top_10_lags = sorted(range(len(autocorrelations)), key=lambda i: abs(autocorrelations[i]), reverse=True)[:10]
# print("Top 10 Lags:", top_10_lags)

In [9]:
cutoff = pd.to_datetime('2024-06-01 00:00:00')

In [ ]:
lags = [i for i in range(1, 1200)]
year_lags = [i for i in range(8700, 8800)]  

lags = lags + year_lags
hours_week = 24 * 7


for lag in lags:
        historical_copy[f"lag{lag}"] = historical_copy['VALUEMWHMETERINGDATA_customerES_5'].shift(lag)

historical_copy['forward_1'] = historical_copy['VALUEMWHMETERINGDATA_customerES_5'].shift(-hours_week)
historical_copy['forward_2'] = historical_copy['VALUEMWHMETERINGDATA_customerES_5'].shift(-2 * hours_week)
historical_copy['forward_3'] = historical_copy['VALUEMWHMETERINGDATA_customerES_5'].shift(-3 * hours_week)
historical_copy['forward_4'] = historical_copy['VALUEMWHMETERINGDATA_customerES_5'].shift(-4 * hours_week)

historical_copy.dropna(inplace=True)

In [26]:
t_mondays = historical_copy[historical_copy.index.dayofweek == 0]
t_tuesdays = historical_copy[historical_copy.index.dayofweek == 1]
t_wednesdays = historical_copy[historical_copy.index.dayofweek == 2]
t_thursdays = historical_copy[historical_copy.index.dayofweek == 3]
t_fridays = historical_copy[historical_copy.index.dayofweek == 4]
t_saturdays = historical_copy[historical_copy.index.dayofweek == 5]
t_sundays = historical_copy[historical_copy.index.dayofweek == 6]

,VALUEMWHMETERINGDATA_customerES_5,forward_1,forward_2,forward_3,forward_4
DATETIME,,,,,
2024-07-29 19:00:00,0.016880,NaN,NaN,NaN,NaN
2024-07-29 20:00:00,0.020584,NaN,NaN,NaN,NaN
2024-07-29 21:00:00,0.023592,NaN,NaN,NaN,NaN
2024-07-29 22:00:00,0.023540,NaN,NaN,NaN,NaN
2024-07-29 23:00:00,0.022128,NaN,NaN,NaN,NaN
